In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

In [2]:
def time_to_seconds(time):
    if ':' in time:
        time = [float(i) for i in time.split(":")]
        return time[0]*60 + time[1]
    else:
        return float(time)

In [3]:
def soup_scores(soup):
    rows = soup.find_all('article', {'class' : 'md:mb-1'})
    quarter = 1
    scores = []
    quarters = []
    times = []
    clocks = []

    for x in np.arange(len(rows)):
        clock = rows[x].find('span', {'class' : 'block'}).text
        game_time = 12*60*4 - (12*60 - time_to_seconds(clock)) - (quarter - 1)*12*60
        
        action = rows[x].find('span', {'class' : 'capitalize'}).text

        if (len(times) > 0) and (game_time > times[-1]):
            quarter += 1
            
        try:
            score = rows[x].find('span', {'class' : "block font-bold"}).text
        except:
            if len(scores) == 0:
                score = "0 - 0"
            else:
                score = scores[-1]
            
        scores.append(score)
        quarters.append(quarter)
        times.append(game_time)
        clocks.append(clock)

    return [scores, quarters, times, clocks]

In [4]:
def final_score(df, team):
    df.loc[len(df)] = [df.SCORE.values[-1],
                       df.Quarter.values[-1],
                       0,
                       "0:00",
                       team,
                       df[df.Team == team].Score.values[-1]]

In [5]:
def score_scraper(url, file):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(url)
    time.sleep(2)
    
    # Accept all cookies in NBA.com if applicable
    WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH,'//*[@id="onetrust-accept-btn-handler"]'))).click()
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    driver.quit()
    
    data = soup_scores(soup)
    score = pd.DataFrame({
        'SCORE': data[0],
        'Quarter': data[1],
        'Time': data[2],
        'Clock': data[3]
    })

    away = score.copy()
    away['Team'] = 'Away'
    away['Score'] = away.SCORE.apply(lambda s : s.split(" - ")[0])

    home = score.copy()
    home['Team'] = 'Home'
    home['Score'] = home.SCORE.apply(lambda s : s.split(" - ")[1])
    
    game = pd.concat([away, home]).sort_index()
    for x in range(10):
        final_score(game, 'Home')
        final_score(game, 'Away')
    game.index = sorted(list(range(int(len(game)/2))) + list(range(int(len(game)/2))))
    
    game.to_csv(file)
    
    return game

In [ ]:
# game = score_scraper("https://www.nba.com/game/bos-vs-gsw-0042100401/play-by-play?period=All", "finals1.csv")
# game = score_scraper("https://www.nba.com/game/bos-vs-gsw-0042100402/play-by-play?period=All", "finals2.csv")
# game = score_scraper("https://www.nba.com/game/bos-vs-gsw-0042100403/play-by-play?period=All", "finals3.csv")
# game = score_scraper("https://www.nba.com/game/bos-vs-gsw-0042100404/play-by-play?period=All", "finals4.csv")
# game = score_scraper("https://www.nba.com/game/bos-vs-gsw-0042100405/play-by-play?period=All", "finals5.csv")
game = score_scraper("https://www.nba.com/game/gsw-vs-bos-0042100406/play-by-play?period=All", "finals6.csv")

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 105.0.5195
[WDM] - Get LATEST chromedriver version for 105.0.5195 google-chrome
[WDM] - There is no [mac64] chromedriver for browser 105.0.5195 in cache
[WDM] - About to download new driver from https://chromedriver.storage.googleapis.com/105.0.5195.52/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/jeremydumalig/.wdm/drivers/chromedriver/mac64/105.0.5195.52]
/var/folders/91/7lxj2xq13852sql471wdl0pw0000gn/T/ipykernel_85954/2414762257.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
game['TIME'] = 2880 - game.Time
game['Lead'] = game.SCORE.apply(lambda s : int(s.split(" - ")[0]) - int(s.split(" - ")[1]))
game['Team'] = game.Lead > 0

game = game.reset_index(drop=False).drop_duplicates(subset=['index'])
game = game.iloc[0:(len(game) - 9)]
game.to_csv("lead.csv")

NameError: name 'game' is not defined